In [1]:
!pip install xlsxwriter

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 kB 5.3 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import glob, os
from time import strftime
from xlsxwriter.utility import xl_rowcol_to_cell
# Database interaction imports
import csv, sqlite3

pd.set_option('display.max_rows', 999)
pd.set_option('display.max_colwidth', 999)
pd.set_option('display.max_columns', 999)
pd.options.display.max_seq_items = 2000
pd.options.display.float_format = '{:20,.2f}'.format


In [3]:
db_file = "/kaggle/input/synthetic-data-sonify/Sonify_DB.db"

con = sqlite3.connect(db_file) 
Dim_Customers_df = pd.read_sql_query("""SELECT * from Dim_Customers""", con)
con.close()
Dim_Customers_df = Dim_Customers_df.head(500)
Dim_Customers_df.head(1)

,Customer_ID,Country,Province,email,FirstName,LastName
0,100000,Canada,Quebec,Michael,Parker,michael.parker@gmail.com


In [4]:
writer = pd.ExcelWriter("RPT_"+strftime("%Y-%m-%d_%H-%M-%S")+".xlsx", engine='xlsxwriter')
Dim_Customers_df.to_excel(writer,index=True, sheet_name='Legend')
Dim_Customers_df.to_excel(writer,index=True, sheet_name='Data')
Dim_Customers_df.to_excel(writer,index=True, sheet_name='Summary')
number_rows = len(Dim_Customers_df.index)

In [5]:
workbook = writer.book
worksheet = writer.sheets['Summary']
worksheet.set_column('A:B', 40)
worksheet.set_column('B:B', 40)
worksheet.write_string('A1', 'COL_NAME')
worksheet.write_string('B1', 'MATCH_COUNTER')

0

In [6]:
workbook = writer.book
worksheet = writer.sheets['Data']
worksheet.set_zoom(100)
worksheet.set_column('A:LM', 30)
worksheet.set_column('LN:RS', 10)

# Define our range for the color formatting
color_range = "LN2:RS{}".format(number_rows+1)

In [7]:
# Add a format. Light red fill with dark red text.
format1 = workbook.add_format({'bg_color': '#FFC7CE',
                               'font_color': '#9C0006'})

# Add a format. Green fill with dark green text.
format2 = workbook.add_format({'bg_color': '#C6EFCE',
                               'font_color': '#006100'})

In [8]:
worksheet.conditional_format(color_range, {'type': 'cell','criteria': '=','value': 'TRUE','format': format2})
worksheet.conditional_format(color_range, {'type': 'cell','criteria': '=','value': 'FALSE','format': format1})

0

In [9]:
worksheet.write_string('A1', 'ACCOUNT_ID')
worksheet.freeze_panes(1, 0)  # Freeze the first row.
worksheet.autofilter('A1:ZZ1')

In [10]:
writer.close()